In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import spacy
import es_core_news_sm

import codecs

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/TFG/textos-reducidos.csv'

# Leer csv donde se encuentran el conjunto de textos de los distintos ODS, cada uno de los atributos esta delimitado por ;; 
df = pd.read_csv(path, sep=";;", engine='python', names=['Numero texto', 'ODS', 'Texto'])
# Poner los ODS con tipo int 
for i in range(len(df.index)):
  df['ODS'][i] = df['ODS'][i][3:]

In [ ]:
# Función que elimina los caracteres que tienen algunos textos del csv al principio 
def clear_first_token(token):
  result = ""
  char = '\ufeff'
  for i in range(len(token)):
    if token[i] != char: result += token[i]
  return result

def preprocess(text):
  # Texto en minusculas
  text_lower = text.lower()
  # Tokenize 
  token_word = nltk.word_tokenize(text_lower, "spanish")
  # Algunos textos tienen caracteres raros al principio, por tanto hay que eliminarlos para que no influyan en el código 
  token_word[0] = clear_first_token(token_word[0])
  # Stopwords de palabras españolas
  stopword_spanish = stopwords.words("spanish")
  i = 0

  while(i < len(token_word)):
    # Se eliminan los tokens que se encuentren dentro de las stopwords 
    if token_word[i] in stopword_spanish:
      token_word.remove(token_word[i])
    #Se elimina cualquier token que sea distinto de caracteres alfanumericos
    elif not (token_word[i].isalpha()):
      token_word.remove(token_word[i])
    else:
      # Debido a que la libreria nltk no lematiza textos en español es necesario utilizar otra libreria de NLP la cual es Spacy
      word = sp(token_word[i])[0]
      token_word[i] = word.lemma_
      i = i+1
  return token_word

In [ ]:
# Tokenize de todos los textos del csv 
sp = es_core_news_sm.load()
df['Tokens']=df['Texto'].apply(preprocess)


In [ ]:
# Se guardan los textos tokenizados en otro csv 
df
df.to_csv("textos_procesados.csv")


In [ ]:
# Unión de los dos csv 
# Leer csv donde se encuentran el conjunto de textos de los distintos ODS, cada uno de los atributos esta delimitado por ;; 

path1 = '/content/drive/MyDrive/TFG/textos_procesados1.csv'
df_procesado = pd.read_csv(path1, usecols=['ODS', 'Texto', 'Tokens'])

path2 = '/content/drive/MyDrive/TFG/textos_procesados2.csv'
df_procesado2 = pd.read_csv(path2, usecols=['ODS', 'Texto', 'Tokens'])

df_final = pd.concat([df_procesado, df_procesado2])
df_final = df_final.reset_index(drop=True)
df_final
df_final.to_csv("textos_procesados_bien.csv")